In [ ]:
print('here...')

In [ ]:
%load_ext autoreload
%autoreload 2

In [2]:
import multibind as mb
import numpy as np
import pandas as pd
import torch
import bindome as bd
bd.constants.ANNOTATIONS_DIRECTORY = 'annotations'
# mb.models.MultiBind
import torch.optim as topti
import torch.utils.data as tdata
import matplotlib.pyplot as plt
import logomaker
import os
import scipy
import pickle

# Use a GPU if available, as it should be faster.
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Using device: " + str(device))

/home/johanna/anaconda3/envs/multibind/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using device: cpu


In [3]:
adata = mb.bindome.datasets.scATAC.PBMCs_10x_v2(datadir='../../atac_poisson_study/data/')
peak_ids = adata.var_names

In [4]:
adata.var['summit'] = ((adata.var['end'] + adata.var['start']) / 2).astype(int)
adata.var['summit.start'] = adata.var['summit'] - 100
adata.var['summit.end'] = adata.var['summit'] + 100

In [5]:
adata.var['k.summit'] = adata.var['chr'] + ':' + adata.var['summit.start'].astype(str) + '-' + adata.var['summit.end'].astype(str)

In [6]:
adata.shape

(10246, 165434)

In [7]:
n_seqs = 1000
seqs = mb.bindome.tl.get_sequences_from_bed(adata.var[['chr', 'summit.start', 'summit.end']].head(n_seqs), genome='hg38', uppercase=True)
keys = set([s[0] for s in seqs])
adata = adata[:,adata.var['k.summit'].isin(keys)]
# seqs = [[s[0], s[1].upper()] for s in seqs[0]]

/tmp/tmp6tapcn2d
genome hg38 False
options
/home/johanna/ICB/annotations/hg38/genome/hg38.fa
True /home/johanna/ICB/annotations/hg38/genome/hg38.fa
running bedtools...
bedtools getfasta -fi /home/johanna/ICB/annotations/hg38/genome/hg38.fa -bed /tmp/tmp6tapcn2d -fo /tmp/tmp36id3l7r


In [8]:
!which bedtools

/usr/bin/bedtools


In [9]:
adata.shape

(10246, 1000)

In [10]:
len(seqs)

1000

In [11]:
# remove Ns
seqs = [[s[0], s[1].replace('N', '')] for s in seqs]
counts = adata.X.T

In [27]:
next_data = pd.DataFrame.sparse.from_spmatrix(counts)

In [28]:
next_data = next_data[range(100)].copy()

In [29]:
next_data['seq'] = [s[1] for s in seqs]

In [30]:
next_data

,0,1,2,3,4,5,6,7,8,9,...,91,92,93,94,95,96,97,98,99,seq
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CCCTAACCCTAACCCTAACCCTAACCCTAACCCCTAACCCTAACCC...
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,ACAACCCTAACCCTAACCCTAACAACCCTAACCCTAACCCTAACCC...
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CGCAGAGAGGCGCGCCGCGCCGGCGCAGGCGCAGAGACACATGCTA...
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,ACTGGGGCCTGAGCTGCGTGGTGGGGAGAGAGTCTCTCCCCTGCCC...
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CTTCTGAAAGGGTTGCTTGACCCACAGATGTGAAGCTGAGGCTGAA...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,GCTGGGCGGTGAGGAGGGAGCTGCTGCCTGGCACGGGGACAGGGTT...
996,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,CTGAAGCCGCTCTCCCCCCACCTCCTGAAAAACGCCAACATTAAAG...
997,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,GCAGCCCCCGGCAGCGCCCACCGGACGGGCTGGAACGGTGGGTTAC...
998,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,TACCTAAGCTGTCCAAGATCTCCCGGCAGCTGCTGACCCCGAGGGA...


In [52]:
# next_data[next_data.columns[:-1]].sum(axis=1).sort_values(ascending=False)

In [31]:
next_data.head(1000).var(axis=1)

/tmp/ipykernel_35357/3781555156.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  next_data.head(1000).var(axis=1)


0      0.010000
1      0.010000
2      0.010000
3      0.000000
4      0.010000
         ...   
995    0.047980
996    0.077172
997    0.082727
998    0.038788
999    0.098889
Length: 1000, dtype: float64

In [54]:
# next_data.var(axis=1)

In [32]:
var = []
for ri, r in next_data.iterrows():
    if ri % 10000 == 0:
        print(ri, next_data.shape)
    # print(ri, r.values[:-1], r.values[:-1].var())
    var.append(r.values[:-1].var())
    # break

0 (1000, 101)


In [33]:
next_data['var'] = var

In [57]:
# sum_index = next_data[next_data.columns[:-1]].var(axis=1).sort_values(ascending=False).index

In [34]:
top_var = next_data[['var']].sort_values('var', ascending=False).index[:1000]

In [35]:
# next_data = next_data.head(10000)
next_data_sel = next_data.reindex(top_var).reset_index(drop=True)

In [36]:
next_data_sel.head()

,0,1,2,3,4,5,6,7,8,9,...,92,93,94,95,96,97,98,99,seq,var
0,2.0,3.0,0.0,3.0,3.0,1.0,7.0,3.0,1.0,1.0,...,1.0,4.0,1.0,1.0,6.0,3.0,1.0,1.0,CGGCCTGGGAAACACTGTAGTGACGCCCAATCAAAACGCCACGTCT...,2.9675
1,3.0,1.0,2.0,2.0,3.0,1.0,7.0,4.0,3.0,3.0,...,3.0,1.0,5.0,0.0,6.0,0.0,3.0,2.0,GCGCCGGGGCGGGGCCTGCGGGCGGGGCGGGGCCCGGGGCCGCGAA...,2.9000
2,0.0,1.0,1.0,2.0,1.0,1.0,3.0,3.0,0.0,0.0,...,0.0,3.0,0.0,2.0,5.0,1.0,3.0,2.0,AGGGGCAGACGGCCATGTTCGTGAACGGCGCTTCCCTTGGTGCACC...,2.5979
3,5.0,0.0,1.0,0.0,1.0,2.0,1.0,1.0,1.0,3.0,...,0.0,1.0,0.0,4.0,7.0,2.0,2.0,1.0,TGTCCGGATTGCATTTCCTCTCTGGAGCTCTCTCTGTCCTGACCTC...,2.1099
4,3.0,1.0,2.0,1.0,2.0,2.0,3.0,0.0,0.0,5.0,...,0.0,3.0,2.0,0.0,3.0,2.0,2.0,0.0,GTTGCTCGTCGGTTCCCTCGGCTCCGCCCCCTCGGCTGTGCGTCTG...,2.0331


In [37]:
del next_data_sel['var']
next_data_sel.index = next_data_sel['seq']
del next_data_sel['seq']

In [38]:
next_data_sel

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
seq,,,,,,,,,,,,,,,,,,,,,
CGGCCTGGGAAACACTGTAGTGACGCCCAATCAAAACGCCACGTCTCGGCCTCGAGGAAATATTCCGTGTCGGGCCCGTCCTGATTGGACAGTTCTTTCGGGTTCTGGCCACTAGGAAGCTTTGTTTAGGTCCGGAAGGCGGGCTTTCCTGGGAGTGGGTGGGGAGGGGGCGTTGATTCTTGACCAATCCTTTCAGTCCG,2.0,3.0,0.0,3.0,3.0,1.0,7.0,3.0,1.0,1.0,...,2.0,2.0,1.0,4.0,1.0,1.0,6.0,3.0,1.0,1.0
GCGCCGGGGCGGGGCCTGCGGGCGGGGCGGGGCCCGGGGCCGCGAATTGGTTAACTCAGCCTGTGGGCGGGGCTGCGCGGTGCGGGGCGGGGCCTTGTGGCGCATGTGAACGCTCATTGGGCCCTCCTGGAAGGGGGCGAGACCAGCGACATAGGGACGGGGCCATGCGGTGGGCGGGTGGAACACGGCGCCGGTAGTGA,3.0,1.0,2.0,2.0,3.0,1.0,7.0,4.0,3.0,3.0,...,1.0,5.0,3.0,1.0,5.0,0.0,6.0,0.0,3.0,2.0
AGGGGCAGACGGCCATGTTCGTGAACGGCGCTTCCCTTGGTGCACCTGTGGCCGCCATGTTTGAGCGAGGTGGGCAACTATGTTCTTGAGGGGGGAGCAACCATGTTCTTGAGGGGCTCTTCCCTTGAAGCCTCTTTGGCCACCATGTTTGAGAGGGGCAGACGGCCATGTTTGTAAGGGGCGCTTCTCTTGGTGCGCCT,0.0,1.0,1.0,2.0,1.0,1.0,3.0,3.0,0.0,0.0,...,1.0,0.0,0.0,3.0,0.0,2.0,5.0,1.0,3.0,2.0
TGTCCGGATTGCATTTCCTCTCTGGAGCTCTCTCTGTCCTGACCTCATTTCCTGTCTGGAGCCAGTGACCTCATTTCCTGTCTGTGTCACTCTGTCCTGACCTCATTTCCTGTCTGTGCCACTCTTTGTCCTGACCTCATTTCCTGTATGCAACACACTGTGCCCGGACTGCATTTCCTGTCTGGAGCTCTCTCTGTCCT,5.0,0.0,1.0,0.0,1.0,2.0,1.0,1.0,1.0,3.0,...,2.0,1.0,0.0,1.0,0.0,4.0,7.0,2.0,2.0,1.0
GTTGCTCGTCGGTTCCCTCGGCTCCGCCCCCTCGGCTGTGCGTCTGGCCCTCAGCTCCACCCCCAGCTGCGCGTCCGTTCCTCAGCTCAGTCCCCAAGCTGTGCGTCCTTCCCTCAGCTCTACCCCCAGCTGTGCGCCCGCCCCTTGGTTCCACCCCCCTCCCCAGCTGTGCGCCCGCCCCTTGGTTCCACCCCCCCCAG,3.0,1.0,2.0,1.0,2.0,2.0,3.0,0.0,0.0,5.0,...,0.0,3.0,0.0,3.0,2.0,0.0,3.0,2.0,2.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
CAGTCCCTCCTCCCTATGGCTCCATCTGGTGGGGACAGAGCAGAGGCCCAGAGGGTTCCAAGACGCAGAGCATGCAGCCAGTGTTAAGGTGGCATCCTGCCCGTCCCGGGTGAGGGGCTCCAGGGAGGAGCAACCTCACCTCAGAACTCCTTGTCCTGCCAGCTCCTGGCCCCTCGTGGCTGCCCCACCACCCTCCTGGC,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
GCACCATTAAAGGTGTACTTTAGGAAGGACTTTGAATGAAGCAATGATGAACAAATAGGCAAACACAAACTGGTAAGAATGTGAGAAGATCTAAAAAAGGATTAAAAATTTAGAAATAAGCATAACAACAAATTTGGGAAAGTAAAAACAAACTGAAACTAAAATACTGGGCAACAGCATCGTGGCAGATGAAAGGGAAG,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
AGGAGGAGGGAGGGAGGCGAGCAGCCAGTCATTCCGGAAGAGCTGCCGGGCCAGACGCAGGGGCTGGGAGAAAGCCAGCTGGCAGGCCCAGTGCTCTCCTTGAGGGCCTGCCCAGGACCACGGCTGGCAGCTGTGTGGGGCTTGGGTCGGGGCTGGGAGGGCTTCTGCTGGCAGCGGGCACCACATTGGAGCCTCGGTGG,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [100]:
df = next_data_sel
dataset = mb.datasets.GenomicsDataset(df)
train = tdata.DataLoader(dataset=dataset, batch_size=256, shuffle=True)

In [101]:
model_atac, best_loss = mb.tl.train_iterative(train, device, num_epochs=500, show_logo=False,
                                         early_stopping=50, log_each=50, n_kernels=2)

next w 15 <class 'int'>
# cells 101

Kernel to optimize 0

Freezing kernels
setting grad status of kernel at 0 to 1
setting grad status of kernel at 1 to 0


kernels mask None
optimizing using <class 'torch.optim.adam.Adam'> and <class 'multibind.tl.loss.MSELoss'> n_epochs 500 early_stopping 50
lr= 0.01, weight_decay= 0.001, dir weight= 0
Epoch: 51, Loss: 176283.604351 , best epoch: 31 secs per epoch: 7.890 s
Epoch: 82, Loss: 176283.6633 , best epoch: 31 secs per epoch: 7.832 s
early stop!
total time: 634.367 s
secs per epoch: 7.832 s

Kernel to optimize 1

Freezing kernels
setting grad status of kernel at 0 to 0
setting grad status of kernel at 1 to 1


kernels mask None
optimizing using <class 'torch.optim.adam.Adam'> and <class 'multibind.tl.loss.MSELoss'> n_epochs 500 early_stopping 50
lr= 0.01, weight_decay= 0.001, dir weight= 0
Epoch: 51, Loss: 0.291397 , best epoch: 49 secs per epoch: 14.582 s
Epoch: 101, Loss: 0.291997 , best epoch: 51 secs per epoch: 14.610 s
Epoch: 102, Loss:

In [104]:
r2 = mb.pl.R2_calculation(model_atac, train)
print("R^2:", r2)

R^2: [-1.07838477e-01 -4.83364150e-01 -1.22575523e-01 -1.86266528e-01
 -6.55286009e-01 -1.91636657e-01  1.14242773e-02 -1.42249611e-01
 -2.03141783e-01 -2.01363935e-02 -3.13520506e-01 -1.99622547e-01
 -1.15867587e+00 -3.66144871e-01 -2.09081271e-01 -4.20815736e-02
 -7.55392481e-02 -3.03890481e-01 -1.44256184e-01 -3.12236872e-01
 -3.50523434e-01 -4.29710304e-01 -1.06089621e-01 -1.27048975e-01
  3.84339951e-03 -3.66993557e-01 -5.33315923e-02 -3.56014660e-01
 -4.79638330e-02 -2.03529243e-01 -4.50742875e-02 -3.35805922e-01
 -2.14879643e-01 -5.78298102e-01 -1.44947804e-01 -1.75238575e-01
 -1.00858233e-01 -3.09273628e-01 -2.21251696e-01 -2.33505850e-01
 -4.86881020e-01 -1.30283465e-01 -3.13618923e-01 -1.97571817e-01
 -6.00388170e-02 -4.01306723e-02 -1.02899442e-01 -8.86769430e-03
 -1.71792531e-01 -4.00070044e-01 -5.39912283e-01 -1.69664298e-01
 -1.79217337e-01 -2.99818352e-01 -1.58237059e-01 -1.36133970e+01
 -8.78090722e-02 -1.46786142e-01 -1.22680398e-02 -4.17124474e-01
 -2.08813071e-01 -2.

In [103]:
model_atac

Multibind(
  (padding): ConstantPad2d(padding=(12, 12, 0, 0), value=0.25)
  (binding_modes): BindingModesPerProtein(
    (generator): BMCollection(
      (conv_mono_list): ModuleList(
        (0): None
        (1): ModuleList(
          (0): Conv2d(1, 1, kernel_size=(4, 15), stride=(1, 1), bias=False)
          (1): Conv2d(1, 1, kernel_size=(4, 15), stride=(1, 1), bias=False)
          (2): Conv2d(1, 1, kernel_size=(4, 15), stride=(1, 1), bias=False)
          (3): Conv2d(1, 1, kernel_size=(4, 15), stride=(1, 1), bias=False)
          (4): Conv2d(1, 1, kernel_size=(4, 15), stride=(1, 1), bias=False)
          (5): Conv2d(1, 1, kernel_size=(4, 15), stride=(1, 1), bias=False)
          (6): Conv2d(1, 1, kernel_size=(4, 15), stride=(1, 1), bias=False)
          (7): Conv2d(1, 1, kernel_size=(4, 15), stride=(1, 1), bias=False)
          (8): Conv2d(1, 1, kernel_size=(4, 15), stride=(1, 1), bias=False)
          (9): Conv2d(1, 1, kernel_size=(4, 15), stride=(1, 1), bias=False)
          (10

In [39]:
df = next_data_sel.copy()
dataset = mb.datasets.GenomicsDataset(df)
train = tdata.DataLoader(dataset=dataset, batch_size=256, shuffle=True)

In [40]:
model_atac, best_loss = mb.tl.train_iterative(train, device, num_epochs=500, show_logo=False,
                                         early_stopping=50, log_each=50, n_kernels=2, joint_learning=True)

next w 15 <class 'int'>
# cells 100

Kernel to optimize 0

Freezing kernels
setting grad status of kernel at 0 to 1
setting grad status of kernel at 1 to 0


kernels mask None
optimizing using <class 'torch.optim.adam.Adam'> and <class 'multibind.tl.loss.MSELoss'> n_epochs 500 early_stopping 50
lr= 0.01, weight_decay= 0.001, dir weight= 0
Epoch: 51, Loss: 183065.608536 , best epoch: 36 secs per epoch: 10.108 s
Epoch: 101, Loss: 183065.639626 , best epoch: 89 secs per epoch: 10.010 s
Epoch: 140, Loss: 183065.5580 , best epoch: 89 secs per epoch: 9.949 s
early stop!
total time: 1382.901 s
secs per epoch: 9.949 s

Kernel to optimize 1

Freezing kernels
setting grad status of kernel at 0 to 0
setting grad status of kernel at 1 to 1


kernels mask None
optimizing using <class 'torch.optim.adam.Adam'> and <class 'multibind.tl.loss.MSELoss'> n_epochs 500 early_stopping 50
lr= 0.01, weight_decay= 0.001, dir weight= 0
Epoch: 51, Loss: 0.232748 , best epoch: 49 secs per epoch: 13.612 s
Epoch: 10

In [41]:
r2 = mb.pl.R2_calculation(model_atac, train)
print("R^2:", r2)

R^2: [ 0.13878875  0.04024839  0.18196623  0.20543248  0.16138362  0.21210567
  0.3165799   0.12787851  0.08510719  0.17636798  0.20414352  0.22971884
  0.02325066  0.21320886  0.08227756  0.13408572  0.15370412  0.10735762
  0.11178474  0.17171434  0.0765277   0.25880751  0.3070136   0.15056937
  0.13319635  0.15720241  0.11522282  0.04697617  0.20400748  0.15545127
  0.18280953  0.1832558   0.14678854  0.12133795  0.15698757  0.21034409
  0.28173869  0.14738089  0.13387148  0.07459993  0.27474017  0.20752409
  0.11371403  0.07238896  0.29358685  0.17234216  0.22219302  0.1688822
  0.21386017  0.11685206  0.0891051   0.1452552   0.19389668  0.12077422
  0.04535193 -5.32148851  0.21553399  0.17289137  0.11450446  0.14526198
  0.18550727  0.23322659  0.03277174  0.24332716  0.21298807  0.17896186
  0.19811828  0.1549659   0.11449261  0.09346913  0.13473171  0.18012241
  0.12753222  0.27793425  0.21873088  0.23055508  0.22269199  0.1351325
  0.20930503  0.17675668  0.18872199  0.09080182